For the problem -u''+ alpha_omega * u = f_omega with Neumann BC

alpha1/2 assumed constants
Steps:
0 - Solve the system with destination psi to set gfud
1- Solve Weak PDE linear system >>>>>> u
2- Solve for adjoint p , EQU 4.8 using u >>>>>>>>>> p

3- Solve for 1st material derivative, EQU 4.5 using u >>>>> u_xo (NxN)
4- Solve for 2nd Adjoint q, EQU 4.20,using u_xo >>>>> q_xo (NxN) 

5- Solve for mat_der of adjoint, EQU 4.9 using u_xo,p >>>>>>>>>>p_xo NOT NEEDED


ORR assemble int(u_xo,u_yo)

In [9]:
import numpy as np
import netgen.geom2d as geom2d
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry  
from ngsolve.internal import *
from interpolations import InterpolateLevelSetToElems # function which interpolates a levelset function

myMAXH = 0.1
EPS = myMAXH * 1e-6      #variable defining when a point is on the interface and when not

geo = SplineGeometry()

R = 2

## add a rectangle
geo.AddRectangle(p1=(-R,-R),
                 p2=(R,R),
                 bc="rectangle",
                 leftdomain=1,
                 rightdomain=0)
geo.SetMaterial (1, "outer") # give the domain the name "outer"
mesh = Mesh(geo.GenerateMesh(maxh=myMAXH)) # generate ngsolve mesh
fes_state = H1(mesh, order=1)
fes_adj = H1(mesh, order=1)
fes_level = H1(mesh, order=1)

pwc = L2(mesh)   #piecewise constant space

N = fes_state.ndof

gfud = GridFunction(fes_state)
gfu = GridFunction(fes_state)

gfu_xo = GridFunction(fes_state)
gfu_yo = GridFunction(fes_state)
gfu_xoyo = GridFunction(fes_state)

gfp = GridFunction(fes_adj)
gfp_xo = GridFunction(fes_adj)
gfq_xo =GridFunction(fes_adj)


psi = GridFunction(fes_level)
psides = GridFunction(fes_level)
psinew = GridFunction(fes_level)


alpha1 = 2
alpha2 = 7
f1=10
f2=1
f_rhs = GridFunction(pwc)
alpha = GridFunction(pwc)

print("Degrees of freedom =",N)

Degrees of freedom = 1942


In [10]:

#STEP 0
#Filling gfud:
a = 4.0/5.0
b = 2
f = CoefficientFunction( 0.1*( (sqrt((x - a)**2 + b * y**2) - 1) \
                * (sqrt((x + a)**2 + b * y**2) - 1) \
                * (sqrt(b * x**2 + (y - a)**2) - 1) \
                * (sqrt(b * x**2 + (y + a)**2) - 1) - 0.001) )

# Setting the piece wise functions according to destination domain
psides.Set(f)
InterpolateLevelSetToElems(psides, f1, f2, f_rhs, mesh, EPS)
InterpolateLevelSetToElems(psides, alpha1, alpha2, alpha, mesh, EPS)

## test and trial functions
u, v = fes_state.TnT()


B = BilinearForm(fes_state)
B += grad(u) * grad(v) * dx
B += alpha* u * v * dx
B.Assemble()

L = LinearForm(fes_state)
L += f_rhs * v *dx
L.Assemble()
inv = B.mat.Inverse( inverse="sparsecholesky") # inverse of bilinear form

gfud.vec.data = inv*L.vec
Draw(gfud)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [11]:
#STEP1 Solving for u

psi.Set(-1) # Initial domain is the holdall
InterpolateLevelSetToElems(psi, f1, f2, f_rhs, mesh, EPS)
InterpolateLevelSetToElems(psi, alpha1, alpha2, alpha, mesh, EPS)

scene_u=Draw(gfu)


def SolvePDE(adjoint=False):

    B.Assemble()
    L.Assemble()

    inv_state = B.mat.Inverse(inverse="sparsecholesky")

    # solve state equation
    gfu.vec.data = inv_state*L.vec
    
    scene_u.Redraw()
    if adjoint == True:
        # solve adjoint state equatoin
        duCost.Assemble()
        B_adj.Assemble()
        inv_adj = B_adj.mat.Inverse(inverse="sparsecholesky")
        gfp.vec.data = -inv_adj * duCost.vec

SolvePDE() #The correct solution for the Neumann conditions is constant everywhere (=f1/alpha1)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [12]:
#STEP2 EQU 4.8
p,q =fes_adj.TnT()
B_adj = BilinearForm(fes_state)
B_adj += grad(p) * grad(q) * dx
B_adj += alpha* p * q * dx
def Cost(u):
    return (u - gfud)**2*dx

duCost = LinearForm(fes_adj)
duCost += 2*(gfu-gfud) * q * dx
SolvePDE(True)
Draw(gfp)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [13]:
#STEPs3,4 _EQU4.5 & 4.20 
gfud_cf = CoefficientFunction(gfud)
gfu_cf = CoefficientFunction(gfu)
#p, q = fes_adj.TnT()
u_x_all = np.zeros((N, N))  # Initialize an NxN matrix
L3 = GridFunction(fes_state)

u_x,v3 = fes_state.TnT()
B3 = BilinearForm(fes_state)
B3 += grad(u_x) * grad(v3) * dx
B3 += alpha* u_x * v3 * dx

#for EQU 4.20

q_x_all = np.zeros((N, N))  # Initialize an NxN matrix

q_x,q4 =fes_adj.TnT()
B4 = BilinearForm(fes_state)
B4 += grad(q_x) * grad(q4) * dx
B4 += alpha* q_x * q4 * dx
#RHS has two terms , a vector matrix multiplication and a vector


phi1,phi2 = fes_adj.TnT()
RHS4= BilinearForm(fes_state)
RHS4+= -2*phi1*phi2*dx
RHS4.Assemble()
RHS4_dense = RHS4.mat.ToDense().NumPy() #NumpyArray
D2J_1st=np.zeros((N,N))
DJ = np.zeros(N)

for i in range(N):
    InterpolateLevelSetToElems(psi, f1, f2, f_rhs, mesh, EPS)
    InterpolateLevelSetToElems(psi, alpha1, alpha2, alpha, mesh, EPS)
    B3.Assemble()
    B4.Assemble()

    L3.Set(0)
    
    sign_x_i = 1 if psi.vec[i] < 0 else -1 
    
    L3.vec[i] = -sign_x_i * (alpha1 - alpha2) * gfu.vec[i] * 1+sign_x_i * (f1 - f2) 
    
    inv3 = B3.mat.Inverse(inverse="sparsecholesky")

    gfu_xo.vec.data = inv3 * L3.vec

    u_x_all[:,i] = gfu_xo.vec
    u_x_cf = CoefficientFunction(gfu_xo) # coefficent function instance of current u_x
    mycf = 2 * (gfu_cf - gfud_cf) * u_x_cf #Coefficient function for DJs RHS
    DJ[i] = Integrate(mycf, mesh)

    
    #RHS of 4.20 is the mass matrix multiplied by u_xo vector
    RHS4_vec = np.zeros(N)
    RHS4_vec = u_x_all[:,i]
    RHS4_total =RHS4_dense @ RHS4_vec #Mass matrix multiply U_xi

    inv4 = B4.mat.Inverse(inverse="sparsecholesky")
    gfq_xo.vec.data = inv4*RHS4_total # solve the system

    q_x_all[:,i] = gfq_xo.vec
            
    for j in range(N):
        sign_x_j = 1 if psi.vec[j] < 0 else -1 
        D2J_1st[i,j]=sign_x_j*(alpha1-alpha2)*gfu.vec[j]*gfq_xo.vec[j]

gdj= GridFunction(fes_state)
gdj.vec.data = DJ
Draw(gdj)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [6]:
# gfu_xi=GridFunction(fes_state)
# gfq_xi=GridFunction(fes_state)
# gfu_xi.vec.data=u_x_all[:,25]
# gfq_xi.vec.data=q_x_all[:,20]
#Draw(gfu_xi)
#Draw(gfq_xi)
gdj= GridFunction(fes_state)
gdj.vec.data = DJ
Draw(gdj)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [14]:
X = np.linalg.solve(D2J_1st, -DJ)
direction = GridFunction(fes_state)
direction.vec.data = X
normDirection = sqrt(Integrate(direction**2*dx, mesh))
direction.vec.data =( 1/normDirection )* direction.vec
Draw(direction)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

for i in range(N):
........
    for j in range(N):
        sign_x_j = 1 if psi.vec[j] < 0 else -1 
        D2J_1st[i,j]=sign_x_j*(alpha1-alpha2)*gfu.vec[j]*gfq_xo.vec[j]

In [9]:

D2J = np.zeros((N, N))

for i in range(N):  # yo
    sign_y_i = 1 if psi.vec[i] < 0 else -1
    for j in range(N):  # xo
        sign_x_j = 1 if psi.vec[j] < 0 else -1 
        D2J[i, j] = sign_x_j * (alpha1 - alpha2) * q_x_all[i, j] * gfu.vec[j] # Correct Term
        D2J[i, j] += sign_x_j * (alpha1 - alpha2) * u_x_all[i, j] * gfp.vec[j]
        D2J[i, j] -= sign_y_i * (alpha1 - alpha2) * u_x_all[j, i] * gfp.vec[i]

XX = np.linalg.solve(D2J, -DJ)

direction2 = GridFunction(fes_state)
direction2.vec.data = XX

Draw(direction2)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [ ]:
norm_direction = sqrt(Integrate(direction**2*dx, mesh)) # L2 norm of TD_node
norm_direction2 = sqrt(Integrate(direction2**2*dx, mesh)) # L2 norm of TD_node
direction.vec.data = 1/norm_direction * direction.vec
direction2.vec.data = 1/norm_direction2 * direction2.vec
#Draw(IfPos(direction,0,1),mesh)
Draw(direction2)
